<a href="https://colab.research.google.com/github/Imposon/Clinical_No_show/blob/main/ai_ml_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joniarroba/noshowappointments")

print("Path to dataset files:", path)

100%|██████████| 2.40M/2.40M [00:00<00:00, 2.67MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/joniarroba/noshowappointments/versions/5


In [2]:
import os

print("Downloaded path:", path)
print("Files inside path:", os.listdir(path))


Downloaded path: /root/.cache/kagglehub/datasets/joniarroba/noshowappointments/versions/5
Files inside path: ['KaggleV2-May-2016.csv']


In [5]:
df = pd.read_csv(os.path.join(path, "KaggleV2-May-2016.csv"))

In [7]:
print("Original shape:", df.shape)
print(df.head())

Original shape: (110527, 14)
      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1  5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2  4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3  8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4  8.841186e+12        5642494      F  2016-04-29T16:07:23Z   

         AppointmentDay  Age      Neighbourhood  Scholarship  Hipertension  \
0  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1   
1  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0   
2  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0   
3  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0   
4  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             1   

   Diabetes  Alcoholism  Handcap  SMS_received No-show  
0         0           0        0             0      No  
1        

In [8]:
df.rename(columns={'No-show':'No_show'}, inplace=True)

In [9]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [10]:
df['lead_time'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
df['appointment_dayofweek'] = df['AppointmentDay'].dt.dayofweek

In [11]:
df = df[df['Age'] >= 0]

In [12]:
df['No_show'] = df['No_show'].map({'Yes':1, 'No':0})